We will be working in the iPython / Jupyter notebook system. I like these because they are a form of literate programming in which we can mix textbook instruction and explanations with code that can also be run and edited.

The text and mathematics in the notebooks requires a little preliminary learning.

### Markdown

You can document your iPython notebooks by making some cells into **Markdown** cells. Markdown is a way of formatting text that is supposed to be almost as readable un-rendered as when it is tidied up. You might argue that it looks equally bad either way, but that's tough because the notebooks use it and that's how I want you to produce nice looking output to hand in as an assignment !

If you look at the **Markdown** cells as source code (by double-clicking on them) you will see how the raw text looks.

### Maths

In a browser, you can render beautiful equations using a javascript tool called **Mathjax** which is build into the iPython notebooks. 

You can build in symbols to your text such as $\pi$ and $\epsilon$ if you use the \$ signs to indicate where your equations begin and end, and you know enough $\LaTeX$ [try it here !](http://www.codecogs.com/latex/eqneditor.php) to get by.

Equations in 'display' mode are written like this

$$ e^{i\pi} + 1 = 0 $$

Go back to the rendered form of the cell by 'running' it.

### Links 

[Markdown Website](http://daringfireball.net/projects/markdown/)

[Mathjax Website](http://docs.mathjax.org)

### Datasets

We need to download some data for the examples. Some files are bundled with the notebooks but some are too large for the repository (and generally, that's not a place to keep anything which is not the primary target of revision management. 

#### Global Magnetic Data

Magnetic intensity data from [geomag.org](http://geomag.org/models/EMAG2/EMAG2_V2.tif)



In [ ]:
## Bundle this up in a helper module 


import requests
import os
import time
import sys

def download_file(url, local_filename):

    r = requests.get(url, stream=True)
             
    start_time = time.time()
    last_time = start_time
    datasize = 0
    
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1000000000): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()
                datasize += len(chunk)
                if (time.time() - last_time) > 2.5:
                    print "{:.1f} Mb in {:.1f}s".format(datasize / 1.0e6, time.time() - start_time)
                    last_time = time.time()

    return 

import hashlib

def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "r+b") as f:
        for block in iter(lambda: f.read(blocksize), ""):
            hash.update(block)
    return hash.hexdigest()


def download_cached_file(location_url, local_file, expected_md5):

    try:
        assert md5sum(local_file) == expected_md5
        print "Using cached file - {}".format(local_file)
        return(2)
    
    except (IOError, AssertionError) as error_info:
        # No file or the wrong file ... best go download it
        # print "Assertion failed - {}".format(sys.exc_info())
        
        try:
            mag_data_file = download_file(location_url, local_file)
            print "Downloaded from {}".format( location_url )
            return(1)

        except:
            print "Unable to download {} [{}] ".format( location_url, sys.exc_info() )
            return(0)



In [ ]:
# Resources required for the tutorial
# The checksum is to make sure any cached file is the right one.
# If the checksum is wrong the file gets downloaded anyway

resource_list = [
# Global magnetic data (120Mb)
{
 "local_file":"Resources/EMAG2_image_V2.tif",
 "md5":"20024c06fdabfa57e4f9efb4aabec198",
 "url":"http://geomag.org/models/EMAG2/EMAG2_image_V2.tif"
},
    
# Blue Marble Image Geotiff (2 Mb)  
{
 "local_file": "Resources/land_shallow_topo_2048.tif",
 "md5": '6390fa98278f6675562b11a6727fc9e5',
 "url":"http://eoimages.gsfc.nasa.gov/images/imagerecords/57000/57752/land_shallow_topo_2048.tif"
},
    
# Blue Marble Image with Bathymetry png (2Mb)
{
 "local_file": "Resources/world.topo.bathy.200409.3x5400x2700.png",
 "md5": 'ef83f9134ab096b0455c30cec926d954',
 "url":"http://eoimages.gsfc.nasa.gov/images/imagerecords/73000/73801/world.topo.bathy.200409.3x5400x2700.png"
},
    

]



In [ ]:

for resource in resource_list:
    print "\nDownloading {:s}".format(resource["local_file"])
    download_cached_file(resource["url"], resource["local_file"], resource["md5"] )



In [ ]:
print md5sum("Images/world.topo.bathy.200409.3x5400x2700.png")